In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
import gc, os
import numpy as np
from tqdm import tqdm
from itertools import combinations

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 200)

In [2]:
train = pd.read_csv('../data/raw/calories.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   User_ID     15000 non-null  int64  
 1   Gender      15000 non-null  object 
 2   Age         15000 non-null  int64  
 3   Height      15000 non-null  float64
 4   Weight      15000 non-null  float64
 5   Duration    15000 non-null  float64
 6   Heart_Rate  15000 non-null  float64
 7   Body_Temp   15000 non-null  float64
 8   Calories    15000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 1.0+ MB


In [4]:
numerical_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

def add_feature_cross_terms(df, numerical_features):
    for f1, f2 in combinations(numerical_features, 2):
        df[f'{f1}_x_{f2}'] = df[f1] * df[f2]

    for f1, f2, f3 in combinations(numerical_features, 3):
            df[f'{f1}_x_{f2}_x_{f3}'] = df[f1] * df[f2] * df[f3]

    return df

def add_interaction_features(df, features):
    df_new = df.copy()
    for f1, f2 in combinations(features, 2):
        df_new[f"{f1}_plus_{f2}"] = df_new[f1] + df_new[f2]
        df_new[f"{f1}_minus_{f2}"] = df_new[f1] - df_new[f2]
        df_new[f"{f2}_minus_{f1}"] = df_new[f2] - df_new[f1]
        df_new[f"{f1}_div_{f2}"] = df_new[f1] / (df_new[f2] + 1e-5)
        df_new[f"{f2}_div_{f1}"] = df_new[f2] / (df_new[f1] + 1e-5)
        
    return df_new

def squares(df, features):
    for feature in features:
        df[f'{feature}_2'] = df[feature]**2

    return df

In [5]:
def preprocesamiento(df):

    # ---- flags & básicos ---------------------------
    df['Gender'] = (df['Gender'] == 'male').astype('int8')
    df['BMI'] = df['Weight'] / (df['Height'] / 100)**2
    df['FCMT_simple'] = 220 - df['Age']
    df['FCMT_tanaka'] = 208 - (0.7 * df['Age'])
    df['Percent_FCMT_simple'] = np.where(df['FCMT_simple'] > 0, (df['Heart_Rate'] / df['FCMT_simple']) * 100, 0)
    df['Percent_FCMT_tanaka'] = np.where(df['FCMT_tanaka'] > 0, (df['Heart_Rate'] / df['FCMT_tanaka']) * 100, 0)
    # Cliping Percent_FCMT a un rango razonable (e.g., 0-150%) para evitar valores extremos si los datos son ruidosos
    df['Percent_FCMT_simple'] = np.clip(df['Percent_FCMT_simple'], 0, 150)
    df['Percent_FCMT_tanaka'] = np.clip(df['Percent_FCMT_tanaka'], 0, 150)
    # Desviación de Temperatura Corporal (Esta SÍ es válida como feature)
    df['Body_Temp_Deviation'] = df['Body_Temp'] - 37.0

    df['Pct_FCMT_sq'] = df['Percent_FCMT_simple']**2
    df['Pct_FCMT_cu'] = df['Percent_FCMT_simple']**3

    # logs
    for col in ['Duration', 'Heart_Rate', 'Body_Temp', 'Weight', 'Duration_x_Heart_Rate']:
        df[f'{col}_log'] = np.log1p(df[col])

    df['is_temp_high']  = (df['Body_Temp']>39).astype('int8')
    df['is_sobrep']  = (df['BMI']>27).astype('int8')

    df['feno_var'] = np.where(df['Gender'] ==1,
                              -55.0969*df['Duration'] + 0.6309*df['Duration_x_Heart_Rate'] + 0.1988*df['Weight_x_Duration'] + 0.2017*df['Age_x_Duration'],
                              -20.4022*df['Duration'] + 0.4472*df['Duration_x_Heart_Rate'] + 0.1263*df['Weight_x_Duration'] + 0.074*df['Age_x_Duration']
                             )

    return df

In [6]:
train = add_feature_cross_terms(train, numerical_features)
train = add_interaction_features(train, numerical_features)
train = squares(train, numerical_features)
preprocesamiento(train)

,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Age_x_Height,Age_x_Weight,Age_x_Duration,Age_x_Heart_Rate,Age_x_Body_Temp,Height_x_Weight,Height_x_Duration,Height_x_Heart_Rate,Height_x_Body_Temp,Weight_x_Duration,Weight_x_Heart_Rate,Weight_x_Body_Temp,Duration_x_Heart_Rate,Duration_x_Body_Temp,Heart_Rate_x_Body_Temp,Age_x_Height_x_Weight,Age_x_Height_x_Duration,Age_x_Height_x_Heart_Rate,Age_x_Height_x_Body_Temp,Age_x_Weight_x_Duration,Age_x_Weight_x_Heart_Rate,Age_x_Weight_x_Body_Temp,Age_x_Duration_x_Heart_Rate,Age_x_Duration_x_Body_Temp,Age_x_Heart_Rate_x_Body_Temp,Height_x_Weight_x_Duration,Height_x_Weight_x_Heart_Rate,Height_x_Weight_x_Body_Temp,Height_x_Duration_x_Heart_Rate,Height_x_Duration_x_Body_Temp,Height_x_Heart_Rate_x_Body_Temp,Weight_x_Duration_x_Heart_Rate,Weight_x_Duration_x_Body_Temp,Weight_x_Heart_Rate_x_Body_Temp,Duration_x_Heart_Rate_x_Body_Temp,Age_plus_Height,Age_minus_Height,Height_minus_Age,Age_div_Height,Height_div_Age,Age_plus_Weight,Age_minus_Weight,Weight_minus_Age,Age_div_Weight,Weight_div_Age,Age_plus_Duration,Age_minus_Duration,Duration_minus_Age,Age_div_Duration,Duration_div_Age,Age_plus_Heart_Rate,Age_minus_Heart_Rate,Heart_Rate_minus_Age,Age_div_Heart_Rate,Heart_Rate_div_Age,Age_plus_Body_Temp,Age_minus_Body_Temp,Body_Temp_minus_Age,Age_div_Body_Temp,Body_Temp_div_Age,Height_plus_Weight,Height_minus_Weight,Weight_minus_Height,Height_div_Weight,Weight_div_Height,Height_plus_Duration,Height_minus_Duration,Duration_minus_Height,Height_div_Duration,Duration_div_Height,Height_plus_Heart_Rate,Height_minus_Heart_Rate,Heart_Rate_minus_Height,Height_div_Heart_Rate,Heart_Rate_div_Height,Height_plus_Body_Temp,Height_minus_Body_Temp,Body_Temp_minus_Height,Height_div_Body_Temp,Body_Temp_div_Height,Weight_plus_Duration,Weight_minus_Duration,Duration_minus_Weight,Weight_div_Duration,Duration_div_Weight,Weight_plus_Heart_Rate,Weight_minus_Heart_Rate,Heart_Rate_minus_Weight,Weight_div_Heart_Rate,Heart_Rate_div_Weight,Weight_plus_Body_Temp,Weight_minus_Body_Temp,Body_Temp_minus_Weight,Weight_div_Body_Temp,Body_Temp_div_Weight,Duration_plus_Heart_Rate,Duration_minus_Heart_Rate,Heart_Rate_minus_Duration,Duration_div_Heart_Rate,Heart_Rate_div_Duration,Duration_plus_Body_Temp,Duration_minus_Body_Temp,Body_Temp_minus_Duration,Duration_div_Body_Temp,Body_Temp_div_Duration,Heart_Rate_plus_Body_Temp,Heart_Rate_minus_Body_Temp,Body_Temp_minus_Heart_Rate,Heart_Rate_div_Body_Temp,Body_Temp_div_Heart_Rate,Age_2,Height_2,Weight_2,Duration_2,Heart_Rate_2,Body_Temp_2,BMI,FCMT_simple,FCMT_tanaka,Percent_FCMT_simple,Percent_FCMT_tanaka,Body_Temp_Deviation,Pct_FCMT_sq,Pct_FCMT_cu,Duration_log,Heart_Rate_log,Body_Temp_log,Weight_log,Duration_x_Heart_Rate_log,is_temp_high,is_sobrep,feno_var
0,14733363,1,68,190.0,94.0,29.0,105.0,40.8,231.0,12920.0,6392.0,1972.0,7140.0,2774.4,17860.0,5510.0,19950.0,7752.0,2726.0,9870.0,3835.2,3045.0,1183.2,4284.0,1214480.0,374680.0,1356600.0,527136.0,185368.0,671160.0,260793.6,207060.0,80457.6,291312.0,517940.0,1875300.0,728688.0,578550.0,224808.0,813960.0,286230.0,111220.8,402696.0,124236.0,258.0,-122.0,122.0,0.357895,2.794117,162.0,-26.0,26.0,0.723404,1.382353,97.0,39.0,-39.0,2.344827,0.426471,173.0,-37.0,37.0,0.647619,1.544117,108.8,27.2,-27.2,1.666666,0.600000,284.0,96.0,-96.0,2.021276,0.494737,219.0,161.0,-161.0,6.551722,0.152632,295.0,85.0,-85.0,1.809524,0.552632,230.8,149.2,-149.2,4.656862,0.214737,123.0,65.0,-65.0,3.241378,0.308511,199.0,-11.0,11.0,0.895238,1.117021,134.8,53.2,-53.2,2.303921,0.434043,134.0,-76.0,76.0,0.276190,3.620688,69.8,-11.8,11.8,0.710784,1.406896,145.8,64.2,-64.2,2.573529,0.388571,4624,36100.0,8836.0,841.0,11025.0,1664.64,26.038781,152,160.4,69.078947,65.461347,3.8,4771.900970,329637.895921,3.401197,4.663439,3.732896,4.553877,8.021585,1,0,1262.9616
1,14861698,0,20,166.0,60.0,14.0,94.0,40.3,66.0,3320.0,1200.0,280.0,1880.0,806.0,9960.0,2324.0,15604.0,6689.8,840.0,5640.0,2418.0,1316.0,564.2,3788.2,199200.0,46480.0,312080.0,133796.0,16800.0,112800.0

In [7]:
train.head()

,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Age_x_Height,Age_x_Weight,Age_x_Duration,Age_x_Heart_Rate,Age_x_Body_Temp,Height_x_Weight,Height_x_Duration,Height_x_Heart_Rate,Height_x_Body_Temp,Weight_x_Duration,Weight_x_Heart_Rate,Weight_x_Body_Temp,Duration_x_Heart_Rate,Duration_x_Body_Temp,Heart_Rate_x_Body_Temp,Age_x_Height_x_Weight,Age_x_Height_x_Duration,Age_x_Height_x_Heart_Rate,Age_x_Height_x_Body_Temp,Age_x_Weight_x_Duration,Age_x_Weight_x_Heart_Rate,Age_x_Weight_x_Body_Temp,Age_x_Duration_x_Heart_Rate,Age_x_Duration_x_Body_Temp,Age_x_Heart_Rate_x_Body_Temp,Height_x_Weight_x_Duration,Height_x_Weight_x_Heart_Rate,Height_x_Weight_x_Body_Temp,Height_x_Duration_x_Heart_Rate,Height_x_Duration_x_Body_Temp,Height_x_Heart_Rate_x_Body_Temp,Weight_x_Duration_x_Heart_Rate,Weight_x_Duration_x_Body_Temp,Weight_x_Heart_Rate_x_Body_Temp,Duration_x_Heart_Rate_x_Body_Temp,Age_plus_Height,Age_minus_Height,Height_minus_Age,Age_div_Height,Height_div_Age,Age_plus_Weight,Age_minus_Weight,Weight_minus_Age,Age_div_Weight,Weight_div_Age,Age_plus_Duration,Age_minus_Duration,Duration_minus_Age,Age_div_Duration,Duration_div_Age,Age_plus_Heart_Rate,Age_minus_Heart_Rate,Heart_Rate_minus_Age,Age_div_Heart_Rate,Heart_Rate_div_Age,Age_plus_Body_Temp,Age_minus_Body_Temp,Body_Temp_minus_Age,Age_div_Body_Temp,Body_Temp_div_Age,Height_plus_Weight,Height_minus_Weight,Weight_minus_Height,Height_div_Weight,Weight_div_Height,Height_plus_Duration,Height_minus_Duration,Duration_minus_Height,Height_div_Duration,Duration_div_Height,Height_plus_Heart_Rate,Height_minus_Heart_Rate,Heart_Rate_minus_Height,Height_div_Heart_Rate,Heart_Rate_div_Height,Height_plus_Body_Temp,Height_minus_Body_Temp,Body_Temp_minus_Height,Height_div_Body_Temp,Body_Temp_div_Height,Weight_plus_Duration,Weight_minus_Duration,Duration_minus_Weight,Weight_div_Duration,Duration_div_Weight,Weight_plus_Heart_Rate,Weight_minus_Heart_Rate,Heart_Rate_minus_Weight,Weight_div_Heart_Rate,Heart_Rate_div_Weight,Weight_plus_Body_Temp,Weight_minus_Body_Temp,Body_Temp_minus_Weight,Weight_div_Body_Temp,Body_Temp_div_Weight,Duration_plus_Heart_Rate,Duration_minus_Heart_Rate,Heart_Rate_minus_Duration,Duration_div_Heart_Rate,Heart_Rate_div_Duration,Duration_plus_Body_Temp,Duration_minus_Body_Temp,Body_Temp_minus_Duration,Duration_div_Body_Temp,Body_Temp_div_Duration,Heart_Rate_plus_Body_Temp,Heart_Rate_minus_Body_Temp,Body_Temp_minus_Heart_Rate,Heart_Rate_div_Body_Temp,Body_Temp_div_Heart_Rate,Age_2,Height_2,Weight_2,Duration_2,Heart_Rate_2,Body_Temp_2,BMI,FCMT_simple,FCMT_tanaka,Percent_FCMT_simple,Percent_FCMT_tanaka,Body_Temp_Deviation,Pct_FCMT_sq,Pct_FCMT_cu,Duration_log,Heart_Rate_log,Body_Temp_log,Weight_log,Duration_x_Heart_Rate_log,is_temp_high,is_sobrep,feno_var
0,14733363,1,68,190.0,94.0,29.0,105.0,40.8,231.0,12920.0,6392.0,1972.0,7140.0,2774.4,17860.0,5510.0,19950.0,7752.0,2726.0,9870.0,3835.2,3045.0,1183.2,4284.0,1214480.0,374680.0,1356600.0,527136.0,185368.0,671160.0,260793.6,207060.0,80457.6,291312.0,517940.0,1875300.0,728688.0,578550.0,224808.0,813960.0,286230.0,111220.8,402696.0,124236.0,258.0,-122.0,122.0,0.357895,2.794117,162.0,-26.0,26.0,0.723404,1.382353,97.0,39.0,-39.0,2.344827,0.426471,173.0,-37.0,37.0,0.647619,1.544117,108.8,27.2,-27.2,1.666666,0.600000,284.0,96.0,-96.0,2.021276,0.494737,219.0,161.0,-161.0,6.551722,0.152632,295.0,85.0,-85.0,1.809524,0.552632,230.8,149.2,-149.2,4.656862,0.214737,123.0,65.0,-65.0,3.241378,0.308511,199.0,-11.0,11.0,0.895238,1.117021,134.8,53.2,-53.2,2.303921,0.434043,134.0,-76.0,76.0,0.276190,3.620688,69.8,-11.8,11.8,0.710784,1.406896,145.8,64.2,-64.2,2.573529,0.388571,4624,36100.0,8836.0,841.0,11025.0,1664.64,26.038781,152,160.4,69.078947,65.461347,3.8,4771.900970,329637.895921,3.401197,4.663439,3.732896,4.553877,8.021585,1,0,1262.9616
1,14861698,0,20,166.0,60.0,14.0,94.0,40.3,66.0,3320.0,1200.0,280.0,1880.0,806.0,9960.0,2324.0,15604.0,6689.8,840.0,5640.0,2418.0,1316.0,564.2,3788.2,199200.0,46480.0,312080.0,133796.0,16800.0,112800.0

In [8]:
train.shape

(15000, 141)

* Crear las funciones de preprocesamiento en features.py
* Guardar los archivos resultantes en data intermedia